In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.stattools import medcouple

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
plt.rcParams["figure.figsize"] = [14, 4]
plt.rcParams["figure.autolayout"] = True

In [ ]:
data_folder = '../data'

In [ ]:
os.listdir(data_folder)

In [ ]:
ofertas_df = pd.read_csv(
    os.path.join(data_folder, 'tabela_ofertas_caneta.csv'),
    parse_dates=['timestamp_evento'],
    encoding='utf-8',
)

In [ ]:
ofertas_df.head(2)

In [ ]:
ofertas_df.info()

# ITEM_PAI

In [ ]:
ofertas_por_item_pai = ofertas_df.groupby(
    by = ['id_item_pai']
).agg([np.mean, np.std, 'count'])
ofertas_por_item_pai.fillna(0, inplace=True)

In [ ]:
ofertas_por_item_pai['preco_da_oferta', 'cv'] = (
    (ofertas_por_item_pai['preco_da_oferta', 'std']/ofertas_por_item_pai['preco_da_oferta', 'mean']) * 100
)
ofertas_por_item_pai.head(2)

In [ ]:
len(ofertas_por_item_pai)

In [ ]:
ofertas_por_item_pai_validas = ofertas_por_item_pai[
  (ofertas_por_item_pai[('preco_da_oferta', 'count')] >= 2) &
  (ofertas_por_item_pai[('preco_da_oferta', 'std')] != 0)
]
ofertas_por_item_pai_validas.head(2)

In [ ]:
len(ofertas_por_item_pai_validas)

In [ ]:
itens_pai_com_variacao_no_preco_ofertado = ofertas_por_item_pai_validas.index.get_level_values('id_item_pai').to_list()

In [ ]:
ofertas_candidatas_a_outliers_considerando_item_pai = ofertas_df.query(
    "id_item_pai in @itens_pai_com_variacao_no_preco_ofertado"
)
ofertas_candidatas_a_outliers_considerando_item_pai.head(2)

# Identificacao de Outliers

## Boxplot

In [ ]:
groupby = ofertas_candidatas_a_outliers_considerando_item_pai.groupby(
    by = ['id_item_pai']
)
boxplot = groupby['preco_da_oferta'].describe()[['mean', 'std', 'count', '25%', '50%', '75%', 'min', 'max']]
indices = groupby.indices

boxplot['iqr'] = boxplot['75%'] - boxplot['25%']
boxplot['limite_inferior'] = boxplot['25%'] - (1.5 * boxplot['iqr'])
boxplot['limite_superior'] = boxplot['75%'] + (1.5 * boxplot['iqr'])
boxplot.head(2)

In [ ]:
outliers = []

with tqdm(total=len(indices.keys())) as pbar:
    for id_item_pai in indices.keys():

        ofertas = ofertas_candidatas_a_outliers_considerando_item_pai.iloc[indices[id_item_pai],:]

        for oferta in ofertas.itertuples():
            if oferta.preco_da_oferta > boxplot.loc[id_item_pai]['limite_superior'] or oferta.preco_da_oferta < boxplot.loc[id_item_pai]['limite_inferior']:
                outliers.append(oferta)
        
        pbar.update(1)
        
outliers_item_pai_boxplot = pd.DataFrame(outliers)
outliers_item_pai_boxplot.drop('Index', axis=1, inplace=True)
outliers_item_pai_boxplot.head(2)

In [ ]:
outliers_item_pai_boxplot.to_csv('../results/outliers_item_pai_ofertas_caneta_boxplot.csv')

## Boxplot ajustado

In [ ]:
boxplot_ajustado = groupby['preco_da_oferta'].agg(
    [
        lambda x : medcouple(x.values), 
        lambda x : np.percentile(x.values, 25),
        lambda x : np.percentile(x.values, 75)
    ]
)

boxplot_ajustado.rename(
    columns={
        '<lambda_0>': 'mc',
        '<lambda_1>': 'q1',
        '<lambda_2>': 'q3',
    },
    inplace=True
)
boxplot_ajustado['iqr'] = boxplot_ajustado['q3'] - boxplot_ajustado['q1']

boxplot_ajustado['limite_inferior'] = boxplot_ajustado.apply(
    lambda x: (x.q1 - (1.5 * np.exp(-4*x.mc) * x.iqr)) if x.mc >= 0 else (x.q1 - (1.5 * np.exp(-3*x.mc) * x.iqr)), 
    axis = 1
)

boxplot_ajustado['limite_superior'] = boxplot_ajustado.apply(
    lambda x: (x.q3 + (1.5 * np.exp(3*x.mc) * x.iqr)) if x.mc >= 0 else (x.q3 + (1.5 * np.exp(4*x.mc) * x.iqr)), 
    axis = 1
)

boxplot_ajustado.head(2)

In [ ]:
outliers = []

with tqdm(total=len(boxplot_ajustado.index)) as pbar:
    for id_item_pai in indices.keys():

        ofertas = ofertas_candidatas_a_outliers_considerando_item_pai.iloc[indices[id_item_pai],:]

        for oferta in ofertas.itertuples():
            if oferta.preco_da_oferta > boxplot_ajustado.loc[id_item_pai]['limite_superior'] or oferta.preco_da_oferta < boxplot_ajustado.loc[id_item_pai]['limite_inferior']:
                outliers.append(oferta)
        
        pbar.update(1)
        
outliers_item_pai_boxplot_ajustado = pd.DataFrame(outliers)
outliers_item_pai_boxplot_ajustado.drop('Index', axis=1, inplace=True)
outliers_item_pai_boxplot_ajustado.head(2)

In [ ]:
outliers_item_pai_boxplot_ajustado.to_csv('../results/outliers_item_pai_ofertas_caneta_boxplot_ajustado.csv')

# Resultados

## Boxplot

In [ ]:
len(outliers_item_pai_boxplot)

## Boxplot ajustado

In [ ]:
len(outliers_item_pai_boxplot_ajustado)